In [5]:
import json
from collections import Counter, defaultdict
import sys
import utils
import pickle
import pandas as pd

In [45]:
partition='full'

In [46]:
input_path='../data/input/%s' % partition
gva_input_annotations_file = '%s/annotation/participants_input.p' % input_path
gva_samefirstname_annotations = '%s/annotation/participants_samefirstname.p' % input_path
gva_samelastname_annotations = '%s/annotation/participants_samelastname.p' % input_path

aux_path='../data/tmp/auxiliary_data'
str_data_path='%s/str_data.json' % aux_path
doc2inc_file='%s/doc2inc.p' % aux_path

### 1. Find participants with multiple incidents

The original purpose of this was to find good examples for participants where the profiling could help establishing identity.

In [20]:
names_set=utils.get_all_names_set(gva_input_annotations_file)
len(names_set)

5329

In [21]:
names_list=utils.get_all_names_list(gva_input_annotations_file)
len(names_list)
cntr=Counter(names_list).most_common()

In [33]:
with open (str_data_path) as f:
    str_data=json.load(f)

In [50]:
len(str_data)

303

In [30]:
part_incidents_pairs=0
for participant_name, freq in cntr:
    participant_name=participant_name.strip()
    incs=set()
    for inc, parts in str_data.items():
        for part in parts:
            if 'Name' in part and part['Name'].strip()==participant_name:
                incs.add(inc)
    if len(incs)>1:
        print(participant_name, incs)
    part_incidents_pairs+=len(incs)

Agent {'650750', '546550'}
Ronald Williams {'588835', '481229', '387194', '510769'}
Trooper {'796051', '635289', '617512'}
Carl Johnson {'448419', '321156'}
Maria Rodriguez {'809499', '572696'}
Omar Hamilton {'296327', '767876'}
Demetrius Davis {'491674', '489061'}
Jorge Hernandez {'625735', '588918'}
David Anderson {'479734', '699605'}
Joel Rodriguez {'723429', '714840'}
Saeve Evans {'710200', '172205'}
Tavarius McKay {'558618', '558601'}
Vincent Wainwright {'409732', '479709'}
Rakim Watson {'588835', '510769', '317165'}
Henry Jackson {'533821', '97407'}
Detective {'791843', '784263'}
Quanisha Sims {'558618', '558601'}
Daniel Munoz {'494072', '384047'}
Matthew Michael Stevenson {'804071', '804189'}
Lawrence Anderson {'732572', '735589'}
Allen Troyer {'502850', '389279'}
Tyquone Greer {'113247', '113214'}
Ismael Moreno Jr. {'156813', '152722'}
Ismael Moreno Jr. {'156813', '152722'}


In [31]:
part_incidents_pairs

697

In [47]:
with open(gva_input_annotations_file, 'rb') as r:
    annotations=pickle.load(r)

In [48]:
for a,b in annotations.items():
    print(a,b)
    break

9f686e8adf67e4f752f6381fb2261260 {'799fa9adeedf4fa64061563b9832e208': {'Name': ' Haylee Garrison', 'Age': ' 8', 'Age Group': ' Child 0-11', 'Gender': ' Female', 'Status': ' Killed', 'Type': ' Victim', 'DeathPlace': 'Illinois', 'DeathDate': '2017'}}


In [49]:
len(annotations.keys()) # how many documents

4479

### [TODO] Fix the data, by: merging the same participants with different names, and by adding BU data

### 2. Compute ambiguity for the various datasets 

In [25]:
print('Ambiguity:')
print(part_incidents_pairs/len(names_set))

Ambiguity:
0.13079376993807468


In [26]:
names_set_samefirst=utils.get_all_names_set(gva_samefirstname_annotations)
len(names_set_samefirst)

3547

In [27]:
print(part_incidents_pairs/len(names_set_samefirst))

0.19650408796165775


In [28]:
names_set_samelast=utils.get_all_names_set(gva_samelastname_annotations)
len(names_set_samelast)

3557

In [29]:
print(part_incidents_pairs/len(names_set_samelast))

0.19595164464436324


### 3. Inspect GVDB

In [ ]:
import pandas as pd
import json

In [ ]:
gvdb_articles_path = "../../gvdb-aggregated-db/Articles-with-extracted-info.tsv"

In [ ]:
#with open(gvdb_articles_path) as f:
gvdb_articles = pd.read_csv(gvdb_articles_path, sep='\t')
#    csv.reader(f, delimiter='\t', quotechar='"')
#    print(len(lines))


In [ ]:
len(gvdb_articles)

In [ ]:
sections=['shooter-section', 'victim-section']

In [ ]:
cnt_attr_values=0
attr='gender'
values=[]
states=[]
dates=[]
for index, row in gvdb_articles.iterrows():
    annotations=json.loads(row['Json'])
    if annotations['date-and-time']['state']:
        states.append(annotations['date-and-time']['state'].strip())
    if annotations['date-and-time']['date']:
        dates.append(annotations['date-and-time']['date'].strip()[:4])
    
    #print(annotations)
    #break
    for sec in sections:
#        print(annotations[sec])
        for part in annotations[sec]:
            if part[attr]:#['value']:
                #print(part[attr]['value'])
                cnt_attr_values+=1
                values.append(part[attr].lower().strip())

In [ ]:
cnt_attr_values

In [ ]:
for v in Counter(values).most_common():
    print(v[0],'\t', v[1])

In [ ]:
for v in Counter(states).most_common():
    print(v[0],'\t', v[1])

In [ ]:
len(states)

In [ ]:
len(dates)

In [ ]:
for v in Counter(dates).most_common():
    print(v[0],'\t', v[1])

### 4. Potential for profiling

In [ ]:
debug=False

In [ ]:
with open(doc2inc_file, 'rb') as f:
    doc2inc = pickle.load(f)

In [ ]:
def unresolvable_pairs_count(participants_file_path):
    with open(participants_file_path, 'rb') as f:
        gva_data = pickle.load(f)
        
    c=0
    nc=0
    for doc_id, data in gva_data.items():
        for participant_id, participant_info in data.items():
            for doc_id2, data2 in gva_samefirstname.items():
                for participant_id2, participant_info2 in data2.items():
                    if participant_id2>participant_id and participant_info==participant_info2:
                        if debug:
                            print(doc2inc[doc_id][0], doc_id, participant_id, participant_info)
                            print(doc2inc[doc_id2][0], doc_id2, participant_id2, participant_info2)
                            print()
                        c+=1
                    elif participant_id2>participant_id:
                        nc+=1
    return c,nc

In [ ]:
count, not_count=unresolvable_pairs_count(gva_input_annotations_file)
print(count, not_count)

In [ ]:
count, not_count=unresolvable_pairs_count(gva_samefirstname_annotations)
print(count, not_count)

In [ ]:
count, not_count=unresolvable_pairs_count(gva_samelastname_annotations)
print(count, not_count)

### 5. Analysis on the entire SE data

In [35]:
se_data_folder='../../semeval18_data'
se_test_data_folder='%s/test_data' % se_data_folder
se_trial_data_folder='%s/trial_data' % se_data_folder
se_frames_file='%s/gva_frames' % se_data_folder

In [36]:
a_df = pd.read_pickle(se_frames_file)

FileNotFoundError: [Errno 2] No such file or directory: '../../semeval18_data/gva_frames'

In [ ]:
len(a_df)

In [ ]:
name2inc=defaultdict(set)
for index, row in a_df.iterrows():
    inc_uri=row['incident_uri']
    for p in row['participants']:
        if 'Name' in p and p['Name']:
            name2inc[p['Name'].strip()].add(inc_uri)

In [ ]:
for name, incs in name2inc.items():
    if len(incs)>1 and name not in ['Officer', '37', 'Deputy', 'Trooper', 'Detective']:
        print(name, len(incs))